In [4]:
%pip install pandas scikit-learn boto3 --quiet

  Using cached jmespath-1.0.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached urllib3-2.3.0-py3-none-any.whl.metadata (6.5 kB)
   ---------------------------------------- 0.0/139.2 kB ? eta -:--:--
   -- ------------------------------------- 10.2/139.2 kB ? eta -:--:--
   ----------------------- ---------------- 81.9/139.2 kB 1.2 MB/s eta 0:00:01
   ---------------------------------------- 139.2/139.2 kB 1.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/13.3 MB ? eta -:--:--
   - -------------------------------------- 0.5/13.3 MB 9.6 MB/s eta 0:00:02
   ---- ----------------------------------- 1.4/13.3 MB 14.8 MB/s eta 0:00:01
   --------- ------------------------------ 3.1/13.3 MB 22.2 MB/s eta 0:00:01
   -------------- ------------------------- 4.9/13.3 MB 28.2 MB/s eta 0:00:01
   --------------------- ------------------ 7.2/13.3 MB 30.5 MB/s eta 0:00:01
   ------------------------- -------------- 8.6/13.3 MB 32.5 MB/s eta 0:00:01
   -----------------------------


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [30]:
import boto3
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

In [28]:
session = boto3.Session()
credentials = session.get_credentials()
current_credentials = credentials.get_frozen_credentials()

In [29]:
session = boto3.Session(
    aws_access_key_id=current_credentials.access_key,
    aws_secret_access_key=current_credentials.secret_key,
    aws_session_token=current_credentials.token
)

In [27]:
curated_bucket = ''
file_key = ''

s3_client = session.c('s3')
response = s3_client.get_object(Bucket=curated_bucket, Key=file_key)
df = pd.read_csv(response['Body'])

ParamValidationError: Parameter validation failed:
Invalid bucket name "": Bucket name must match the regex "^[a-zA-Z0-9.\-_]{1,255}$" or be an ARN matching the regex "^arn:(aws).*:(s3|s3-object-lambda):[a-z\-0-9]*:[0-9]{12}:accesspoint[/:][a-zA-Z0-9\-.]{1,63}$|^arn:(aws).*:s3-outposts:[a-z\-0-9]+:[0-9]{12}:outpost[/:][a-zA-Z0-9\-]{1,63}[/:]accesspoint[/:][a-zA-Z0-9\-]{1,63}$"

In [ ]:

numerical_cols = car_data.select_dtypes(include=['float64', 'int64']).columns
imputer = SimpleImputer(strategy='median')
car_data[numerical_cols] = imputer.fit_transform(car_data[numerical_cols])
car_data.head()

In [ ]:


# Load data from the curated S3 bucket


# Preprocess data for training
df.dropna(subset=['price'], inplace=True)
X = df.drop(columns=['price'])
y = df['price']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a model
model = LinearRegression()
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

# Save the model
import joblib
joblib.dump(model, 'car_price_model.pkl')

NoSuchBucket: An error occurred (NoSuchBucket) when calling the GetObject operation: The specified bucket does not exist